# Data Prep

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from joblib import dump
from scipy.sparse import save_npz

from data_prep_func import prep_process_save_data

In [2]:
SEED = 123

## Preprocessing Functions

In [3]:
def encode_ip_none(df):
    """Drop IP address fields completely."""
    df = df.copy()
    return df.drop(columns=["src_ip", "dst_ip"]), None

def encode_ip_integer(df):
    """Convert IPv4 x.x.x.x to an integer: a*256^3 + b*256^2 + c*256 + d."""
    def ip_to_int(ip):
        try:
            a, b, c, d = map(int, ip.split("."))
            return (a << 24) + (b << 16) + (c << 8) + d
        except:
            return 0
    df = df.copy()
    df["src_ip_int"] = df["src_ip"].apply(ip_to_int)
    df["dst_ip_int"] = df["dst_ip"].apply(ip_to_int)

    return df.drop(columns=["src_ip", "dst_ip"]), ["src_ip_int", "dst_ip_int"]

IP_ENCODERS = {
    "none": encode_ip_none,
    "integer": encode_ip_integer,
}

In [4]:
def prepare_data(df,feature_set, ip_encoding="none"):
    '''
    Prepare data by filtering on selected features.

    Args:
        df: Input DataFrame with flow data.
        feature_set: List of features to retain.
        ip_encoding: IP address encoding method ("none", "integer", "onehot").
    Returns:
        df: Prepared DataFrame with selected features.
        numeric_cols: List of numeric feature column names.
        categorical_cols: List of categorical feature column names.
        ip_feature_cols: List of IP feature column names.
    '''
    # Drop metadata fields
    df = df.drop(columns=["flow_id", "attack_id", "phase", "attack"])

    # Handle IP address fields
    if ip_encoding not in IP_ENCODERS:
        raise ValueError(f"Unknown IP encoding: {ip_encoding}")

    df, ip_feature_cols = IP_ENCODERS[ip_encoding](df)

    # Categorical features
    categorical_cols = [
        "proto",      # categorical: "tcp", "udp", "icmp", etc.
        "service",    # categorical: "http", "ftp", "dns", etc.
        "conn_state", # categorical: "S0", "S1", "SF", etc.
        "local_orig", # binary flags
        "local_resp", # binary flags
        ]
    
    # Numerical features
    numeric_cols = [
        "start_time",
        "end_time",
        "duration", 
        "sport",
        "dport",
        "orig_bytes", 
        "resp_bytes",
        "orig_pkts", 
        "resp_pkts",
    ]

    # Filter on features
    categorical_cols = [col for col in categorical_cols if col in feature_set]
    numeric_cols = [col for col in numeric_cols if col in feature_set]

    ip_feature_cols = ip_feature_cols if ip_feature_cols else []
    numeric_cols.extend(ip_feature_cols)

    df = df[feature_set + ip_feature_cols]

    return df, numeric_cols, categorical_cols, ip_feature_cols

In [5]:
def construct_pipeline(numeric_cols, categorical_cols):
    '''
    Construct a preprocessing pipeline for numerical and categorical features.
    Args:
        numeric_cols: List of numerical feature column names.
        categorical_cols: List of categorical feature column names.
    Returns:
        pipeline: sklearn Pipeline object for preprocessing. 
    '''
    transformer = ColumnTransformer(
        transformers=[
            ("numerical", StandardScaler(with_mean=False), numeric_cols),
            ("categorical", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_cols),
        ], 
        sparse_threshold = 1.0 # always return sparse matrix
    )
    pipeline = Pipeline(steps=[("transform", transformer)])

    return pipeline

In [6]:
def save_processed_data(X_train, y_train, X_test, y_test, pipeline, numeric_cols, categorical_cols, ip_encoding, output_dir):
    '''
    Save processed data and preprocessing pipeline to disk.
    Args:
        X_train: Processed training feature matrix.
        y_train: Training labels.
        X_test: Processed testing feature matrix.
        y_test: Testing labels.
        pipeline: Preprocessing pipeline.
        numeric_cols: List of numerical feature column names.
        categorical_cols: List of categorical feature column names.
        ip_encoding: IP encoding method used.
        output_dir: Directory to save processed data.
    Returns:
        None
    '''
    os.makedirs(output_dir, exist_ok=True)

    np.save(os.path.join(output_dir, "y_train.npy"), y_train)
    np.save(os.path.join(output_dir, "y_test.npy"), y_test)
    save_npz(os.path.join(output_dir, "X_train.npz"), X_train)
    save_npz(os.path.join(output_dir, "X_test.npz"), X_test)

    dump(pipeline, os.path.join(output_dir, "feature_pipeline.joblib"))

    with open(os.path.join(output_dir, "feature_info.txt"), "w") as f:
        f.write("Numerical features:\n")
        f.write(str(numeric_cols) + "\n\n")
        f.write("Categorical features:\n")
        f.write(str(categorical_cols) + "\n\n")
        f.write(f"IP encoding: {ip_encoding}\n")

    print(f"Saved X, y, and preprocessing pipeline to {output_dir}/")

## Choice of Features

In [7]:
ip_encoding = "none"
# ip_encoding = "integer"

In [8]:
# all features 
# feature_set = ["start_time", "end_time", "duration", "sport", "dport", "proto", "orig_bytes", "resp_bytes", "orig_pkts", "resp_pkts"]

# without time
# feature_set = ["duration", "sport", "dport", "proto", "orig_bytes", "resp_bytes", "orig_pkts", "resp_pkts"]

# without ports
# feature_set = ["start_time", "end_time", "duration", "proto", "orig_bytes", "resp_bytes", "orig_pkts", "resp_pkts"]

# without time and ports
feature_set = ["duration", "proto", "orig_bytes", "resp_bytes", "orig_pkts", "resp_pkts"]


## Scenario One

In [9]:
SCENARIO_ONE_INSIDE_CSV = "../../data/DARPA_2000/Scenario_One/inside/inside_labeled_flows_all.csv"
SCENARIO_ONE_DMZ_CSV = "../../data/DARPA_2000/Scenario_One/dmz/dmz_labeled_flows_all.csv"

In [10]:
scenario_one_dir = "processed_data/scenario_one"

### Split Mode: Inside (Stratified Split)

In [11]:
data_out_root_dir = f"{scenario_one_dir}/inside/stratified"

In [12]:
# Load labeled data
df = pd.read_csv(SCENARIO_ONE_INSIDE_CSV)
df.head()

,flow_id,start_time,end_time,duration,src_ip,sport,dst_ip,dport,proto,service,orig_bytes,resp_bytes,orig_pkts,resp_pkts,conn_state,local_orig,local_resp,attack_id,attack,phase
0,f0,9.524389e+08,9.524389e+08,0.101800,172.16.112.50,33354,172.16.114.50,80,tcp,http,290,7897,8,9,SF,T,T,0,0,0
1,f1,9.524389e+08,9.524389e+08,0.001107,172.16.114.50,45135,172.16.115.20,53,udp,dns,44,131,1,1,SF,T,T,0,0,0
2,f2,9.524389e+08,9.524389e+08,0.064488,172.16.113.204,22641,172.16.112.100,25,tcp,smtp,623,247,12,22,SF,T,T,0,0,0
3,f3,9.524389e+08,9.524389e+08,0.001167,172.16.112.100,1061,172.16.115.20,53,udp,dns,90,131,2,1,SF,T,T,0,0,0
4,f4,9.524389e+08,9.524389e+08,0.000861,172.16.113.204,1438,172.16.115.20,53,udp,dns,45,131,1,1,SF,T,T,0,0,0


In [13]:
# Split Data into Train/Test
train_size = 0.6
test_size = 1 - train_size
df_train, df_test = train_test_split(
    df, test_size=test_size, stratify=df["phase"], random_state=SEED
)

print("Train set shape:", df_train.shape)
print("Test set shape:", df_test.shape)

Train set shape: (75495, 20)
Test set shape: (50330, 20)


In [14]:
df_train_features, numeric_cols, categorical_cols, ip_feature_cols = prepare_data(
    df_train, feature_set, ip_encoding=ip_encoding, 
)
df_test_features, _, _, _ = prepare_data(
    df_test, feature_set, ip_encoding=ip_encoding,
)

print("Numeric cols:", numeric_cols)
print("Categorical cols:", categorical_cols)
print("IP feature cols:", ip_feature_cols)

df_train_features.head()

Numeric cols: ['duration', 'orig_bytes', 'resp_bytes', 'orig_pkts', 'resp_pkts']
Categorical cols: ['proto']
IP feature cols: []


,duration,proto,orig_bytes,resp_bytes,orig_pkts,resp_pkts
30301,0.000797,udp,35,95,1,1
121252,0.113402,tcp,186,29495,13,24
103980,0.000000,tcp,0,0,1,0
112109,0.000000,tcp,0,0,1,0
83648,0.000000,tcp,0,0,1,0


In [15]:
pipeline = construct_pipeline(numeric_cols, categorical_cols)

X_train = pipeline.fit_transform(df_train_features)
y_train = df_train["attack"]
X_test = pipeline.transform(df_test_features)
y_test = df_test["attack"]

print("--- Training Data ---")
print("Feature matrix shape:", X_train.shape)
print("Labels shape:", y_train.shape)

print("--- Test Data ---")
print("Feature matrix shape:", X_test.shape)
print("Labels shape:", y_test.shape)

--- Training Data ---
Feature matrix shape: (75495, 8)
Labels shape: (75495,)
--- Test Data ---
Feature matrix shape: (50330, 8)
Labels shape: (50330,)


In [16]:
# Save processed data
save_processed_data(
    X_train, y_train, X_test, y_test, 
    pipeline, numeric_cols, categorical_cols, ip_encoding,
    output_dir=f"{data_out_root_dir}/unbalanced")


Saved X, y, and preprocessing pipeline to processed_data/scenario_one/inside/stratified/unbalanced/


### Split Mode: "insidedmz"

In [17]:
data_out_root_dir = f"{scenario_one_dir}/inside_dmz"

In [18]:
# Load data (inside and dmz)
df_inside = pd.read_csv(SCENARIO_ONE_INSIDE_CSV)
print("Inside set shape:", df_inside.shape)
# df_inside.head()

df_dmz = pd.read_csv(SCENARIO_ONE_DMZ_CSV)
print("DMZ set shape:", df_dmz.shape)
df_dmz.head()

Inside set shape: (125825, 20)
DMZ set shape: (45441, 20)


,flow_id,start_time,end_time,duration,src_ip,sport,dst_ip,dport,proto,service,orig_bytes,resp_bytes,orig_pkts,resp_pkts,conn_state,local_orig,local_resp,attack_id,attack,phase
0,f0,9.524389e+08,9.524389e+08,0.071043,172.16.112.50,43703,172.16.114.50,80,tcp,http,285,5952,7,18,SF,T,T,0,0,0
1,f1,9.524389e+08,9.524389e+08,0.008217,172.16.114.50,1965,172.16.115.20,53,udp,dns,88,131,2,1,SF,T,T,0,0,0
2,f2,9.524389e+08,9.524389e+08,0.021633,172.16.113.204,43704,199.95.74.90,80,tcp,http,235,406,5,5,SF,T,F,0,0,0
3,f3,9.524389e+08,9.524389e+08,0.067433,172.16.113.204,43705,199.95.74.90,80,tcp,http,236,17635,8,17,SF,T,F,0,0,0
4,f4,9.524389e+08,9.524389e+08,0.020937,172.16.113.204,43706,199.95.74.97,80,tcp,http,297,282,5,4,SF,T,F,0,0,0


In [19]:
df_train = df_inside.copy()
df_test = df_dmz.copy()

print("Train set shape:", df_train.shape)
print("Test set shape:", df_test.shape)

Train set shape: (125825, 20)
Test set shape: (45441, 20)


In [20]:
prep_process_save_data(
    df_train, df_test, feature_set, ip_encoding, 
    output_dir=f"{data_out_root_dir}/unbalanced"
)

Saved X, y, and preprocessing pipeline to processed_data/scenario_one/inside_dmz/unbalanced/


## Scenario Two

In [21]:
SCENARIO_TWO_INSIDE_CSV = "../../data/DARPA_2000/Scenario_Two/inside/inside_labeled_flows_all.csv"
SCENARIO_TWO_DMZ_CSV = "../../data/DARPA_2000/Scenario_Two/dmz/dmz_labeled_flows_all.csv"

scenario_two_dir = "processed_data/scenario_two"


### Split Mode: Inside (Stratified Split)

In [22]:
data_out_root_dir = f"{scenario_two_dir}/inside/stratified"

In [23]:
# Load labeled data
df = pd.read_csv(SCENARIO_TWO_INSIDE_CSV)
df.head()

,flow_id,start_time,end_time,duration,src_ip,sport,dst_ip,dport,proto,service,orig_bytes,resp_bytes,orig_pkts,resp_pkts,conn_state,local_orig,local_resp,attack_id,attack,phase
0,f0,9.559107e+08,9.559107e+08,0.195749,172.16.112.149,2104,195.73.151.50,79,tcp,finger,8,141,6,5,SF,T,F,0,0,0
1,f1,9.559107e+08,9.559107e+08,0.011438,172.16.112.149,1472,172.16.115.20,53,udp,dns,35,94,1,1,SF,T,T,0,0,0
2,f2,9.559107e+08,9.559107e+08,0.003681,172.16.115.20,32780,192.168.1.50,53,udp,dns,35,94,1,1,SF,T,T,0,0,0
3,f3,9.559107e+08,9.559107e+08,0.119006,172.16.113.207,2105,207.25.71.200,80,tcp,http,261,24153,11,20,SF,T,F,0,0,0
4,f4,9.559107e+08,9.559107e+08,0.027521,172.16.113.207,2106,207.25.71.200,80,tcp,http,329,2445,5,5,SF,T,F,0,0,0


In [24]:
# Split Data into Train/Test
train_size = 0.6
test_size = 1 - train_size
df_train, df_test = train_test_split(
    df, test_size=test_size, stratify=df["phase"], random_state=SEED
)

print("Train set shape:", df_train.shape)
print("Test set shape:", df_test.shape)

Train set shape: (47265, 20)
Test set shape: (31510, 20)


In [25]:
prep_process_save_data(
    df_train, df_test, feature_set, ip_encoding, 
    output_dir=f"{data_out_root_dir}/unbalanced"
)

Saved X, y, and preprocessing pipeline to processed_data/scenario_two/inside/stratified/unbalanced/


## Both Scenarios

In [26]:
scenarios_dir = "processed_data/both_scenarios"

### Split Mode: "inside"

Train on scenario one (inside traffic), test on scenario two (inside traffic).

In [27]:
data_out_root_dir = f"{scenarios_dir}/inside"

In [28]:
# Load Data 
df_scenario_one_inside = pd.read_csv(SCENARIO_ONE_INSIDE_CSV)
print("Inside set shape:", df_scenario_one_inside.shape)

df_scenario_two_inside = pd.read_csv(SCENARIO_TWO_INSIDE_CSV)
print("Inside set shape:", df_scenario_two_inside.shape)


Inside set shape: (125825, 20)
Inside set shape: (78775, 20)


In [29]:
df_train = df_scenario_one_inside.copy()
df_test = df_scenario_two_inside.copy()

In [30]:
prep_process_save_data(
    df_train, df_test, feature_set, ip_encoding, 
    output_dir=f"{data_out_root_dir}/unbalanced"
)

Saved X, y, and preprocessing pipeline to processed_data/both_scenarios/inside/unbalanced/
